 LangGraph


Project Title

Intelligent Customer Support Ticket Automation using LangGraph and Hugging Face

1. Business Problem

Modern organizations receive a very large number of customer support tickets through:

Emails

Chatbots

Web forms

Social media

These tickets vary in:

Topic (billing, technical issue, account access, etc.)

Urgency (normal vs critical)

Complexity

Current Challenges

Manual ticket triage is slow and costly

Support agents waste time reading long messages

High-priority tickets may be missed

Response quality varies between agents

2. Project Objective

The goal of this project is to automate the initial handling of customer support tickets using AI, so that:

Tickets are automatically understood

Important issues are prioritized

Support agents receive summaries and suggested replies

Overall response time and cost are reduced

The system must:

Work without paid APIs

Be scalable and modular

Use open-source models

3. Why LangGraph + Hugging Face?
Hugging Face

Provides pre-trained and fine-tunable NLP models

Supports tasks like classification, summarization, and text generation

Can run locally, avoiding external APIs

LangGraph

Orchestrates AI workflows as a graph

Maintains state across multiple steps

Supports branching, decision-making, and memory

Ideal for multi-step business logic (not just single prompts)

Together, they create a production-ready AI pipeline.

4. High-Level System Architecture

The system is designed as a multi-stage AI workflow, not a single model call.

Input

A raw customer ticket (text)

Processing Pipeline

Each step is handled by a dedicated AI model and coordinated by LangGraph.

Output

Ticket category

Urgency level

Short summary

Suggested response

5. Workflow Explanation (Step-by-Step)
Step 1: Ticket Ingestion

The system receives a customer message

This could come from email, chat, or a helpdesk system

The raw text is stored in the workflow state

Step 2: Ticket Classification

A text classification model analyzes the ticket

It determines the ticket type (e.g., Billing, Technical, General)

This information helps route the ticket to the correct team

Business value:
Reduces manual sorting and misrouting

Step 3: Urgency Detection

Another model checks if the ticket is urgent

Keywords and context are analyzed (e.g., “service down”, “payment failed”)

Urgent tickets are flagged for priority handling

Business value:
Critical issues are addressed faster

Step 4: Ticket Summarization

Long customer messages are condensed into 2–3 lines

The summary highlights the core problem

Agents no longer need to read the full message

Business value:
Improves agent efficiency and reduces cognitive load

Step 5: Suggested Response Generation

A language generation model creates a draft reply

The response is polite, contextual, and relevant

Agents can edit or approve it before sending

Business value:
Speeds up responses while maintaining quality

Step 6: Final Output

The system produces a structured result:

Category

Urgency flag

Summary

Suggested reply

This output is sent to:

A support dashboard

A CRM system

Or directly to a human agent

6. Role of LangGraph in the Project

LangGraph:

Connects all steps as nodes in a graph

Passes information (state) between nodes

Ensures the workflow runs in the correct order

Allows future branching (e.g., auto-escalation if urgent)

Without LangGraph, this would be hard-coded logic.
With LangGraph, it becomes flexible, readable, and scalable.

7. Model Usage Strategy

Each task uses the most suitable model, instead of one large model doing everything.

Classification model → understanding intent

Summarization model → condensing information

Generation model → drafting responses

This improves:

Accuracy

Speed

Resource efficiency

8. Scalability and Extensibility

This project can be easily extended to:

Multilingual support

Automatic escalation to managers

Sentiment analysis

Feedback-based re-training

Analytics dashboards for ticket trends

9. Business Impact
Operational Benefits

Faster ticket handling

Reduced manual effort

Lower support costs

Customer Benefits

Faster responses

More consistent communication

Better issue resolution

Technical Benefits

No vendor lock-in

Fully open-source

Works offline or on-premise

10. Summary

This project demonstrates how LangGraph and Hugging Face can be combined to build a real-world AI system that goes beyond simple chatbots.

It shows:

Practical AI orchestration

Responsible use of open-source models

Clear business value

In short, it is a production-ready AI workflow for intelligent customer support automation.

In [13]:
!pip install -r /content/sample_data/requirements.txt


In [15]:
pip install langchain-huggingface


In [17]:
# =========================
# Imports
# =========================
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
from transformers import pipeline


# =========================
# Define State
# =========================
class TicketState(TypedDict):
    ticket_text: str
    category: str
    summary: str
    suggested_reply: str


# =========================
# Load Hugging Face Models
# =========================

# 1. Classification model
classifier = pipeline(
    "text-classification",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

# 2. Summarization model
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn"
)

# 3. Text generation model
generator = pipeline(
    "text-generation",
    model="gpt2"
)


# =========================
# LangGraph Nodes
# =========================

def classify_ticket(state: TicketState) -> TicketState:
    result = classifier(state["ticket_text"])[0]
    state["category"] = result["label"]
    return state


def summarize_ticket(state: TicketState) -> TicketState:
    summary = summarizer(
        state["ticket_text"],
        max_length=60,
        min_length=25,
        do_sample=False
    )[0]["summary_text"]
    state["summary"] = summary
    return state


def suggest_response(state: TicketState) -> TicketState:
    prompt = f"Customer issue: {state['summary']}\nSupport response:"
    response = generator(
        prompt,
        max_length=80,
        num_return_sequences=1
    )[0]["generated_text"]
    state["suggested_reply"] = response
    return state


# =========================
# Build LangGraph
# =========================
graph = StateGraph(TicketState)

# Add nodes WITH NAMES
graph.add_node("classify", classify_ticket)
graph.add_node("summarize", summarize_ticket)
graph.add_node("respond", suggest_response)

# Add edges USING NODE NAMES
graph.add_edge(START, "classify")
graph.add_edge("classify", "summarize")
graph.add_edge("summarize", "respond")
graph.add_edge("respond", END)

# Compile graph
app = graph.compile()


# =========================
# Run the Workflow
# =========================
input_state = {
    "ticket_text": "My internet has been down since morning and I cannot work."
}

output = app.invoke(input_state)

print("\n===== FINAL OUTPUT =====\n")
print("Category:", output["category"])
print("\nSummary:", output["summary"])
print("\nSuggested Reply:\n", output["suggested_reply"])


Device set to use cpu
Device set to use cpu
Device set to use cpu
Your max_length is set to 60, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gener


===== FINAL OUTPUT =====

Category: NEGATIVE

Summary: "My internet has been down since morning and I cannot work. My internet is down," she said. "I cannot work"

Suggested Reply:
 Customer issue: "My internet has been down since morning and I cannot work. My internet is down," she said. "I cannot work"
Support response: "Please help us to find the source of the problem."
The council confirmed it was investigating the incident and that the issue had been resolved.
A spokesman for the city said: "We are in contact with the CPS and are working closely with the public to try and resolve this issue."
